In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest,chi2,f_regression,r_regression,mutual_info_regression, f_classif

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,LeaveOneOut,ShuffleSplit,StratifiedKFold,GroupKFold
from sklearn.tree import DecisionTreeRegressor,plot_tree

from sklearn.linear_model import LinearRegression,RidgeCV,LassoCV,ElasticNetCV

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,mean_absolute_percentage_error,max_error 

import warnings
warnings.filterwarnings('ignore')

In [2]:
absence=['na','-','I']
bokhol=pd.read_excel('basedonnees.xlsx',na_values=absence,parse_dates=True,index_col=[0])

FileNotFoundError: [Errno 2] No such file or directory: 'basedonnees.xlsx'

In [ ]:
bokhol.info()


: 

In [ ]:
bokhol=bokhol.dropna(axis=0)
bokhol.isnull().sum()

: 

In [ ]:
bokhol=bokhol.drop([bokhol.columns[1],bokhol.columns[2]],axis=1)
bokhol.head()

: 

In [ ]:
X=bokhol.drop([bokhol.columns[4],bokhol.columns[2]],axis=1)
y=bokhol[['Production Centrale Meter 1 - (W)']]

: 

In [ ]:
seuil=12 # POUR 1H
X_train=X[12:]
y_train=y[12:]

X_teste=X[:12]
y_teste=y[:12]

print(X_train.shape)
print(X_teste.shape)
print("______________________________")
print(y_train.shape)
print(y_teste.shape)

: 

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

params={'n_neighbors':np.arange(1,20),'metric':['euclidean','manhattan']}
grid=GridSearchCV(KNeighborsRegressor(),params,cv=10)
grid.fit(X_train,y_train)
modele4=grid.best_estimator_

print(f"score sur le train set:{modele4.score(X_train,y_train)}")
print("_________performances estatistique apres entrainement_________")
print(f"RMSE_train:{np.sqrt(mean_squared_error(y_train,modele4.predict(X_train)))}")
print(f"MAE_train:{mean_absolute_error(y_train, modele4.predict(X_train))}")
print(f"MAPE_train:{np.sqrt(mean_absolute_percentage_error(y_train,modele4.predict(X_train)))}")
print(f"ME_train:{max_error(y_train,modele4.predict(X_train))}")
print("__________performances statistiques en test___________________")
print(f"score sur le teste set:{modele4.score(X_teste,y_teste)}")
print(f"MAE_test:{mean_absolute_error(y_teste, modele4.predict(X_teste))}")


: 

In [ ]:
scal=MinMaxScaler()
X_traina=scal.fit_transform(np.array(X_train))
y_traina=scal.fit_transform(np.array(y_train))

X_testea=scal.fit_transform(np.array(X_teste))
y_testea=scal.fit_transform(np.array(y_teste))

: 

In [ ]:
params={'n_neighbors':np.arange(1,20),'metric':['euclidean','manhattan']}
grid=GridSearchCV(KNeighborsRegressor(),params,cv=10)
grid.fit(X_traina,y_traina)
modele4a=grid.best_estimator_

print(f"score sur le train set:{modele4a.score(X_train,y_train)}")
print("_________performances estatistique apres entrainement_________")
print(f"RMSE_train:{np.sqrt(mean_squared_error(y_traina,modele4a.predict(X_traina)))}")
print(f"MAE_train:{mean_absolute_error(y_traina, modele4a.predict(X_traina))}")
print(f"MAPE_train:{np.sqrt(mean_absolute_percentage_error(y_traina,modele4a.predict(X_traina)))}")
print(f"ME_train:{max_error(y_traina,modele4a.predict(X_traina))}")
print("__________performances statistiques en test___________________")
print(f"score sur le teste set:{modele4.score(X_teste,y_teste)}")
print(f"MAE_test:{mean_absolute_error(y_teste, modele4.predict(X_teste))}")

: 

In [ ]:
Sn=['MSE_train','MAE_train','ME_train']
sv=[2096316.0230640965,1118186.953551498,1118186.953551498]
plt.figure(figsize=(15,7))
plt.bar(x=Sn,height=sv)
plt.title("diagramme en barre des erreurs statistique")
plt.grid()

: 

In [ ]:
N,train_score,val_score=learning_curve(modele4,X_train,y_train,
                                       train_sizes=np.linspace(0.1,1.0,20),cv=5)
plt.plot(N,train_score.mean(axis=1),label='train')
plt.plot(N,val_score.mean(axis=1),label='validation')
plt.grid()
plt.legend();

: 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_train,label='serie brute')
plt.plot(X_train.index,modele4.predict(X_train),label='prediction_train')
plt.legend();

: 

In [ ]:
modele4.predict(X_train).shape
modele4.predict(X_train).ravel().shape

: 

In [ ]:
data_predictKNN=pd.DataFrame({'DATE':y_train.index,
              'y_train':np.array(y_train).ravel(),
              'predict':modele4.predict(X_train).ravel()
              })

data_predictKNN.set_index('DATE',inplace=True)

: 

In [ ]:
data_predictKNN

: 

In [ ]:
mois=['2021-01-01','2021-03-01','2021-06-01','2021-09-01','2021-12-01']
moisch=['jenvier','Mars','juin','septembre','Decembre']
plt.figure(figsize=(15,10))
for i,j in enumerate(mois):
    plt.subplot(3,3,1+i)
    plt.plot(data_predictKNN[j])
    plt.title(moisch[i])

: 

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(data_predictKNN['2021-03-01'])
plt.title('Mars')

: 

In [ ]:
data_predictKNN['2021-01-01'].plot(figsize=(15,5))
plt.title('janvier');

: 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_teste,label='serie brute',lw=3)
plt.plot(X_teste.index,modele4.predict(X_teste),label='prediction_train')
plt.legend()

: 

### ensemble methode Adaboost

In [ ]:
from sklearn.ensemble import BaggingRegressor
# Création du modèle
model = BaggingRegressor(random_state=42)
# Définition de la grille de paramètres pour la recherche
param_grid = {
    'n_estimators': [50, 100, 200,300],
    #'learning_rate': [0.01, 0.1,0.5,0.6,0.7,0.8, 1.0]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10)
grid_search.fit(X_traina, y_traina)
# Affichage des meilleurs paramètres
print("Best Parameters: ", grid_search.best_params_)
# Prédiction et évaluation du modèle
best_model = grid_search.best_estimator_
# Sur l'ensemble d'entrainement
train_predictions = best_model.predict(X_traina)
print("R2_Train:", r2_score(y_traina, train_predictions))
# Sur l'ensemble de test
#test_predictions = best_model.predict(X_teste)
#print("R2_Test:", r2_score(y_teste, test_predictions))

: 

In [ ]:
print(f"MSE_train:{np.sqrt(mean_squared_error(y_traina,best_model.predict(X_traina)))}")
print(f"MAE_train:{mean_absolute_error(y_traina, best_model.predict(X_traina))}")
print(f"MAPE_train:{np.sqrt(mean_absolute_percentage_error(y_traina,best_model.predict(X_traina)))}")
print(f"ME_train:{max_error(y_traina,best_model.predict(X_traina))}")
#print("__________performances statistiques en test___________________")
#print(f"score sur le teste set:{best_model.score(X_teste,y_teste)}")
#print(f"MAE_test:{mean_absolute_error(y_teste, best_model.predict(X_teste))}")

: 

In [ ]:
Sn=['MSE_train','MAE_train','ME_train']
sv=[814005.9143290258,431997.36353347125,20869642.92857143]
plt.figure(figsize=(15,7))
plt.bar(x=Sn,height=sv)
plt.title("diagramme en barre des erreurs statistique bagging")
plt.grid()

: 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_train,label='train')
plt.plot(X_train.index,best_model.predict(X_train),label='predict');
plt.legend();

: 

In [ ]:
bm=best_model.predict(X_train)
bm

: 

In [ ]:
data_predict=pd.DataFrame({'DATE':y_train.index,
              'y_train':np.array(y_train).ravel(),
              'predict':bm})



: 

In [ ]:
data_predict.set_index('DATE',inplace=True)
data_predict

: 

In [ ]:
mois=['2021-01-01','2021-03-01','2021-06-01','2021-09-01','2021-12-01']
moisch=['jenvier','Mars','juin','septembre','Decembre']
plt.figure(figsize=(15,10))
for i,j in enumerate(mois):
    plt.subplot(3,3,1+i)
    plt.plot(data_predict[j])
    plt.title(moisch[i])

: 

In [ ]:
mois=['2021-01-01','2021-03-01']
moisch=['jenvier','Mars']
plt.figure(figsize=(15,6))
for i,j in enumerate(mois):
    plt.subplot(1,2,1+i)
    plt.plot(data_predict[j])
    plt.title(moisch[i])

: 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_teste,label='TEST',lw=5)
plt.plot(X_teste.index,best_model.predict(X_teste),label='predict',lw=3)
plt.legend()

: 

### LINEAR MODEL

In [ ]:
modele_1=LinearRegression()
modele_1.fit(X_train,y_train)
trat=KFold(10)#,shuffle=False, random_state=1)
#print(cross_val_score(modele_1,X_train,y_train,cv=trat).mean())

print(f"R2_train:{modele_1.score(X_train,y_train)}")
print(f"R2_teste:{modele_1.score(X_teste,y_teste)}")
print(f"MSE_TRAIN:{np.sqrt(mean_squared_error(y_train,modele_1.predict(X_train)))}")
print(f"MSE_TESTE:{np.sqrt(mean_squared_error(y_teste,modele_1.predict(X_teste)))}")

: 

In [ ]:
COEF=modele_1.coef_.tolist()[0]
plt.figure(figsize=(15,5))
plt.bar(x=X.columns,height=COEF)
plt.title('les parametres du modele')
plt.ylabel('coeffocients')
plt.grid()

: 

In [ ]:
modele_1.predict(X_teste).shape

: 

In [ ]:
N,train_score,val_score=learning_curve(modele_1,X_train,y_train,
                                       train_sizes=np.linspace(0.1,1.0,20),cv=5)
plt.plot(N,train_score.mean(axis=1),label='train')
plt.plot(N,val_score.mean(axis=1),label='validation')
plt.grid()
plt.legend()

: 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_train,label='serie brute')
plt.plot(X_train.index,modele_1.predict(X_train),label='prediction_train')
plt.legend()

: 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_teste,label='serie brute test')
plt.plot(X_teste.index,modele_1.predict(X_teste),label='prediction_test')
plt.legend()

: 

In [ ]:
# Créer des modèles de régression Ridge, Lasso et ElasticNet avec validation croisée
ridge_model = RidgeCV(alphas=np.logspace(-5, 5, 20), cv=10)
lasso_model = LassoCV(alphas=np.logspace(-5, 5, 20), cv=10)
enet_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas=np.logspace(-5, 5, 20), cv=10)

# Ajuster les modèles aux données d'apprentissage
ridge_model.fit(X_train, y_train)
lasso_model.fit(X_train, y_train)
enet_model.fit(X_train, y_train)
# Calculer le R2 et le RMSE pour les données d'apprentissage et de test
ridge_train_r2 = r2_score(y_train, ridge_model.predict(X_train))
ridge_test_r2 = r2_score(y_teste, ridge_model.predict(X_teste))
ridge_train_rmse = np.sqrt(mean_squared_error(y_train,ridge_model.predict(X_train)))
ridge_test_rmse = np.sqrt(mean_squared_error(y_teste, ridge_model.predict(X_teste)))

lasso_train_r2 = r2_score(y_train, lasso_model.predict(X_train))
lasso_test_r2 = r2_score(y_teste, lasso_model.predict(X_teste))
lasso_train_rmse = np.sqrt(mean_squared_error(y_train,lasso_model.predict(X_train)))
lasso_test_rmse = np.sqrt(mean_squared_error(y_teste,lasso_model.predict(X_teste)))

enet_train_r2 = r2_score(y_train, enet_model.predict(X_train))
enet_test_r2 = r2_score(y_teste, enet_model.predict(X_teste))
enet_train_rmse = np.sqrt(mean_squared_error(y_train,enet_model.predict(X_train)))
enet_test_rmse = np.sqrt(mean_squared_error(y_teste,enet_model.predict(X_teste)))
data = {'Modèle': ['Ridge', 'Lasso', 'ElasticNet'],
        'R2 sur base train': [ridge_train_r2, lasso_train_r2, enet_train_r2],
'R2 sur base test': [ridge_test_r2, lasso_test_r2, enet_test_r2],
'RMSE sur base train': [ridge_train_rmse, lasso_train_rmse, enet_train_rmse],
'RMSE sur base test': [ridge_test_rmse, lasso_test_rmse, enet_test_rmse]}

df = pd.DataFrame(data)
df

: 

### Reseau de neurone

In [ ]:
import tensorflow as tf

model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=15,activation='relu',
                                kernel_initializer='uniform',
                                input_dim=3))
model.add(tf.keras.layers.Dense(units=1,activation='relu'))

model.compile(optimizer='adam',loss='mse',metrics=['mse'])


: 

In [ ]:
model1=tf.keras.Sequential()
model1.add(tf.keras.layers.Dense(units=15,activation='relu',
                                kernel_initializer='uniform',
                                input_dim=3))
model1.add(tf.keras.layers.Dense(units=15,activation='relu',
                                kernel_initializer='uniform',
                                input_dim=3))
model1.add(tf.keras.layers.Dense(units=1,activation='relu'))
model1.compile(optimizer='adam',loss='mse',metrics=['mse'])

: 

In [ ]:
model1.summary()

: 

In [ ]:
print(model.summary())

: 

In [ ]:
scal=MinMaxScaler()
X_train_scal=scal.fit_transform(X_train)
X_teste_scal=scal.fit_transform(X_teste)
y_train_scal=scal.fit_transform(y_train)
y_teste_scal=scal.fit_transform(y_teste)

: 

In [ ]:
model.fit(X_train_scal,y_train_scal,epochs=50)

: 

In [ ]:
model.evaluate(X_teste_scal,y_teste_scal)

: 

In [ ]:
print(f'testR2:{r2_score(y_teste_scal,model.predict(X_teste_scal))}')
print(f'trainR2:{r2_score(y_train_scal,model.predict(X_train_scal))}')

: 

In [ ]:
MSE_train=np.sqrt(mean_squared_error(scal.inverse_transform(y_train_scal),
                                     scal.inverse_transform(model.predict(X_train_scal))))
MSE_teste=np.sqrt(mean_squared_error(scal.inverse_transform(y_teste_scal),
                                     scal.inverse_transform(model.predict(X_teste_scal))))
print(f'mse train:{MSE_train}')
"""
print(f"MAE_train:{mean_absolute_error(scal.inverse_transform(y_train_scal),
      scal.inverse_transform(model.predict(X_train_scal)))}")
#print(f"MAPE_train:{np.sqrt(mean_absolute_percentage_error(scal.inverse_transform(y_train_scal),
                                     #scal.inverse_transform(model.predict(X_train_scal))))}")
#print(f"ME_train:{max_error(scal.inverse_transform(y_train_scal),
                                     #scal.inverse_transform(model.predict(X_train_scal)))}")"""

: 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(X_train.index,y_train_scal,label='serie brute')
plt.plot(X_train.index,model.predict(X_train_scal),label='prediction_train')
plt.legend()

: 

In [ ]:
PMC=model.predict(X_train_scal)

: 

In [ ]:
data_predictPMC=pd.DataFrame({'DATE':y_train.index,
              'y_train':np.array(y_train_scal).ravel(),
              'predict':PMC.ravel()})
data_predictPMC.set_index('DATE',inplace=True)

: 

In [ ]:
mois=['2021-01-01','2021-03-01','2021-06-01','2021-09-01','2021-12-01']
moisch=['jenvier','Mars','juin','septembre','Decembre']
plt.figure(figsize=(15,7))
for i,j in enumerate(mois):
    plt.subplot(3,3,1+i)
    plt.plot(data_predictPMC[j])
    plt.title(moisch[i])

: 

In [ ]:
mois=['2021-01-01','2021-03-01']
moisch=['jenvier','Mars']
plt.figure(figsize=(15,5))
for i,j in enumerate(mois):
    plt.subplot(1,2,1+i)
    plt.plot(data_predictPMC[j])
    plt.title(moisch[i])

: 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(X_teste.index,y_teste_scal,label='serie brute',lw=3)
plt.plot(X_teste.index,model.predict(X_teste_scal),label='prediction_train')
plt.legend()

: 

In [ ]:
model1.fit(X_train_scal,y_train_scal,epochs=50)

: 

In [ ]:
print(f'testR2:{r2_score(y_teste_scal,model1.predict(X_teste_scal))}')
print(f'trainR2:{r2_score(y_train_scal,model1.predict(X_train_scal))}')

: 

In [ ]:
x=np.linspace(-4,4,100)
f=np.exp(-x**2)
plt.figure(figsize=(15,7))
plt.plot(x,f)
plt.title('La fonction gaussienne')

: 